In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from numpy import mean , concatenate
from math import sqrt
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense,LSTM,Activation,Dropout,TimeDistributed,Flatten, Bidirectional, Reshape
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf

from sklearn.externals import joblib

In [ ]:
tf.random.set_seed(2021)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data import and first clean

In [ ]:
#ex. path1 = 'Bangkok', path2 = 'Bangkok'
def get_data(path1,path2):
  wind_train = pd.read_csv('/content/drive/Shareddrives/DS/DS_kaggle_edited/'+path1+'/Train/3H_wind_'+path1+'.csv')
  temp_train = pd.read_csv('/content/drive/Shareddrives/DS/DS_kaggle_edited/'+path1+'/Train/3H_temperature_'+path1+'.csv')
  pm_train = pd.read_csv('/content/drive/Shareddrives/DS/DS_kaggle_edited/'+path1+'/Train/'+path2+'DT.csv')

  wind_test = pd.read_csv('/content/drive/Shareddrives/DS/DS_kaggle_edited/'+path1+'/Test/3H_wind_'+path1+'.csv')
  temp_test = pd.read_csv('/content/drive/Shareddrives/DS/DS_kaggle_edited/'+path1+'/Test/3H_temperature_'+path1+'.csv')
  pm_test = pd.read_csv('/content/drive/Shareddrives/DS/DS_kaggle_edited/'+path1+'/Test/'+path1+'.csv')

  #Train Data
  pm_train['date'] = pd.to_datetime(pm_train['datetime'])
  pm_train = pm_train.set_index('date')
  pm_train = pm_train[['PM2.5']]
  pm_train_dp = pm_train[~pm_train.index.duplicated(keep='first')]
  pm_train_dp = pm_train_dp.asfreq('H')

  wind_train['date'] = pd.to_datetime(wind_train['datetime'])
  wind_train = wind_train.set_index('date')
  wind_train = wind_train[['Wind Speed(km/h)','WindDir']]
  wind_train_dp = wind_train[~wind_train.index.duplicated(keep='first')]
  wind_train_dp = wind_train_dp.asfreq('H')
  wind_train_dp['WindDir'] = wind_train_dp['WindDir'].replace(360.0,0.0)

  temp_train['date'] = pd.to_datetime(temp_train['datetime'])
  temp_train = temp_train.set_index('date')
  temp_train = temp_train[['Temp(C)']]
  temp_train_dp = temp_train[~temp_train.index.duplicated(keep='first')]
  temp_train_dp = temp_train_dp.asfreq('H')

  train_data = pm_train_dp
  train_data['Temp'] = temp_train_dp['Temp(C)']
  train_data['WindDir'] = wind_train_dp['WindDir']
  train_data['WindSpd'] = wind_train_dp['Wind Speed(km/h)']
  train_data.loc['2019-03-18 07:00:00'] = [np.NaN,np.NaN,np.NaN,np.NaN]

  train_data = train_data.interpolate()
  train_data = train_data.fillna(method='bfill')
  train_data = train_data.asfreq('H')

  #Test Data
  wind_test['date'] = pd.to_datetime(wind_test['datetime'])
  wind_test = wind_test.set_index('date')
  wind_test = wind_test[['Wind Speed(km/h)','WindDir']]
  wind_test_dp = wind_test[~wind_test.index.duplicated(keep='first')]
  wind_test_dp = wind_test_dp.asfreq('H')
  wind_test_dp['WindDir'] = wind_test_dp['WindDir'].replace(360.0,0.0)

  temp_test['date'] = pd.to_datetime(temp_test['datetime'])
  temp_test = temp_test.set_index('date')
  temp_test = temp_test[['Temp(C)']]
  temp_test_dp = temp_test[~temp_test.index.duplicated(keep='first')]
  temp_test_dp = temp_test_dp.asfreq('H')

  pm_test = pm_test.rename(columns = {'Unnamed: 0': 'date','PM2.5(µg/m3)':'PM2.5'}, inplace = False)
  pm_test = pm_test.set_index('date')
  pm_test = pm_test.asfreq('H')
  pm_test_dp = pm_test.fillna(method='ffill')

  test_data = pm_test_dp
  test_data['Temp'] = temp_test_dp['Temp(C)']
  test_data['WindDir'] = wind_test_dp['WindDir']
  test_data['WindSpd'] = wind_test_dp['Wind Speed(km/h)']
  test_data = test_data.interpolate()
  test_data = test_data.fillna(method='bfill')

  all_data = pd.concat([train_data, test_data])
  all_data = all_data.asfreq('H')
  return train_data, test_data, all_data

In [ ]:
chan_train, chan_test, chan_all = get_data('Chanthaburi', 'Chanthaburi')
cm_train, cm_test, cm_all = get_data('Chiang Mai','Chiang_Mai')
kan_train, kan_test, kan_all = get_data('Kanchanaburi','Kanchanaburi')
bkk_train, bkk_test, bkk_all = get_data('Bangkok','Bangkok')
kon_train, kon_test, kon_all = get_data('Khon Kaen','Khon_Kaen')
song_train, song_test, song_all = get_data('Songkhla','Songkhla')

merge dataframe

In [ ]:
all_df = pd.DataFrame()
all_df[['Temp_chan','WindDir_chan','WindSpd_chan']] = chan_all[['Temp','WindDir','WindSpd']]
all_df[['Temp_cm','WindDir_cm','WindSpd_cm']] = cm_all[['Temp','WindDir','WindSpd']]
all_df[['Temp_kan','WindDir_kan','WindSpd_kan']] = kan_all[['Temp','WindDir','WindSpd']]
all_df[['Temp_bkk','WindDir_bkk','WindSpd_bkk']] = bkk_all[['Temp','WindDir','WindSpd']]
all_df[['Temp_kon','WindDir_kon','WindSpd_kon']] = kon_all[['Temp','WindDir','WindSpd']]
all_df[['Temp_song','WindDir_song','WindSpd_song']] = song_all[['Temp','WindDir','WindSpd']]
all_df['PM2.5_chan'] =  chan_all['PM2.5']
all_df['PM2.5_cm'] =  cm_all['PM2.5']
all_df['PM2.5_kan'] =  kan_all['PM2.5']
all_df['PM2.5_bkk'] =  bkk_all['PM2.5']
all_df['PM2.5_kon'] =  kon_all['PM2.5']
all_df['PM2.5_song'] =  song_all['PM2.5']
all_df

,Temp_chan,WindDir_chan,WindSpd_chan,Temp_cm,WindDir_cm,WindSpd_cm,Temp_kan,WindDir_kan,WindSpd_kan,Temp_bkk,WindDir_bkk,WindSpd_bkk,Temp_kon,WindDir_kon,WindSpd_kon,Temp_song,WindDir_song,WindSpd_song,PM2.5_chan,PM2.5_cm,PM2.5_kan,PM2.5_bkk,PM2.5_kon,PM2.5_song
date,,,,,,,,,,,,,,,,,,,,,,,,
2016-03-03 15:00:00,31.700000,55.000000,10.000000,33.4,230.000000,5.000000,36.700000,85.000000,10.000000,34.600000,60.000000,12.000000,33.000000,80.0,6.000000,29.300000,95.000000,31.000000,44.2,53.7,65.2,62.9,35.800000,31.2
2016-03-03 16:00:00,31.700000,55.000000,10.000000,33.4,230.000000,5.000000,36.700000,85.000000,10.000000,34.600000,60.000000,12.000000,33.000000,80.0,6.000000,29.300000,95.000000,31.000000,44.2,53.7,65.2,62.9,35.800000,31.2
2016-03-03 17:00:00,30.200000,58.333333,12.333333,30.4,226.666667,6.666667,35.066667,90.000000,11.000000,32.766667,63.333333,11.666667,31.366667,95.0,6.666667,28.433333,91.666667,30.333333,44.3,53.7,64.9,55.5,36.000000,31.2
2016-03-03 18:00:00,28.700000,61.666667,14.666667,27.4,223.333333,8.333333,33.433333,95.000000,12.000000,30.933333,66.666667,11.333333,29.733333,110.0,7.333333,27.566667,88.333333,29.666667,44.2,53.7,64.9,55.5,35.900000,31.2
2016-03-03 19:00:00,27.200000,65.000000,17.000000,24.4,220.000000,10.000000,31.800000,100.000000,13.000000,29.100000,70.000000,11.000000,28.100000,125.0,8.000000,26.700000,85.000000,29.000000,34.2,33.4,38.4,47.9,38.033333,24.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-18 16:00:00,29.400000,90.000000,8.000000,38.9,250.000000,27.000000,37.700000,160.000000,3.000000,33.000000,210.000000,7.000000,33.500000,240.0,13.000000,29.600000,100.000000,31.000000,19.4,53.9,40.9,23.9,48.300000,11.3
2020-03-18 17:00:00,28.633333,86.666667,9.666667,35.8,250.000000,25.666667,35.600000,166.666667,7.666667,32.133333,226.666667,6.666667,33.166667,240.0,15.333333,28.566667,100.000000,30.333333,19.8,46.5,33.4,24.4,42.000000,11.7
2020-03-18 18:00:00,27.866667,83.333333,11.333333,32.7,250.000000,24.333333,33.500000,173.333333,12.333333,31.266667,243.333333,6.333333,32.833333,240.0,17.666667,27.533333,100.000000,29.666667,22.9,42.9,28.4,24.9,42.700000,11.7


In [ ]:
all_train_df = all_df.loc[:'2019-03-18 07:00:00']
all_test_df = all_df.loc['2019-03-18 07:00:00':]

In [ ]:
len_train = len(chan_train)
len_test  = len(chan_test)

#Load Saved model

In [ ]:
scaler = []
scaler_path = '/content/drive/Shareddrives/DS/Model Export/scaler_' # + i.save

#load scaler
for i in range(24):
  scaler.append(joblib.load(scaler_path+str(i)+'.save'))

[MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(0, 1)),
 MinMaxScaler(copy=True, feature_range=(

In [ ]:
json_file = open('/content/drive/Shareddrives/DS/Model Export/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/Shareddrives/DS/Model Export/model.h5")
print("Loaded model from disk")

loaded_model.compile(loss="mse", optimizer="adam")
loaded_model.summary()

Loaded model from disk
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 48, 200)           100000    
_________________________________________________________________
dense_10 (Dense)             (None, 48, 50)            10050     
_________________________________________________________________
dense_11 (Dense)             (None, 48, 30)            1530      
_________________________________________________________________
dense_12 (Dense)             (None, 48, 9)             279       
_________________________________________________________________
reshape_3 (Reshape)          (None, 72, 6)             0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 72, 6)             42        
Total params: 111,901
Trainable params: 111,901
Non-trainable params: 0
_________________________

# Normalize

In [ ]:
all_scaled_df_val = all_df.copy(True)
all_scaled_df_val = all_scaled_df_val.values

for i in range(24):
  all_scaled_df_val[:,[i]] = scaler[i].fit_transform(all_scaled_df_val[:,[i]])
all_scaled_df_val = np.asarray(all_scaled_df_val).astype(np.float32)
all_scaled_df_val

array([[0.82978725, 0.15492958, 0.12195122, ..., 0.54506063, 0.04543724,
        0.34705228],
       [0.82978725, 0.15492958, 0.12195122, ..., 0.54506063, 0.04543724,
        0.34705228],
       [0.75      , 0.16431925, 0.15040651, ..., 0.48093587, 0.04569108,
        0.34705228],
       ...,
       [0.6258865 , 0.23474178, 0.13821138, ..., 0.21577123, 0.0541947 ,
        0.13014461],
       [0.5851064 , 0.22535211, 0.15853658, ..., 0.23396881, 0.05901764,
        0.12124583],
       [0.5851064 , 0.22535211, 0.15853658, ..., 0.22616984, 0.06498287,
        0.13348165]], dtype=float32)

# Data cleanup for using as a exog for prediction

In [ ]:
n_steps_in, n_steps_out, n_features = 24 , 72 , 24

In [ ]:
all_scaled_df = pd.DataFrame(all_scaled_df_val)
all_scaled_df.index = all_df.index
all_scaled_df.columns = all_df.columns
all_scaled_df

,Temp_chan,WindDir_chan,WindSpd_chan,Temp_cm,WindDir_cm,WindSpd_cm,Temp_kan,WindDir_kan,WindSpd_kan,Temp_bkk,WindDir_bkk,WindSpd_bkk,Temp_kon,WindDir_kon,WindSpd_kon,Temp_song,WindDir_song,WindSpd_song,PM2.5_chan,PM2.5_cm,PM2.5_kan,PM2.5_bkk,PM2.5_kon,PM2.5_song
date,,,,,,,,,,,,,,,,,,,,,,,,
2016-03-03 15:00:00,0.829787,0.154930,0.121951,0.672043,0.647887,0.031847,0.814815,0.239437,0.104167,0.816901,0.169014,0.142857,0.706250,0.225352,0.070588,0.555556,0.267606,0.373494,0.444221,0.263494,0.425865,0.545061,0.045437,0.347052
2016-03-03 16:00:00,0.829787,0.154930,0.121951,0.672043,0.647887,0.031847,0.814815,0.239437,0.104167,0.816901,0.169014,0.142857,0.706250,0.225352,0.070588,0.555556,0.267606,0.373494,0.444221,0.263494,0.425865,0.545061,0.045437,0.347052
2016-03-03 17:00:00,0.750000,0.164319,0.150407,0.591398,0.638498,0.042463,0.759820,0.253521,0.114583,0.730829,0.178404,0.138889,0.655208,0.267606,0.078431,0.486772,0.258216,0.365462,0.445226,0.263494,0.423906,0.480936,0.045691,0.347052
2016-03-03 18:00:00,0.670213,0.173709,0.178862,0.510753,0.629108,0.053079,0.704826,0.267606,0.125000,0.644757,0.187793,0.134921,0.604167,0.309859,0.086275,0.417989,0.248826,0.357430,0.444221,0.263494,0.423906,0.480936,0.045564,0.347052
2016-03-03 19:00:00,0.590426,0.183099,0.207317,0.430108,0.619718,0.063694,0.649832,0.281690,0.135417,0.558685,0.197183,0.130952,0.553125,0.352113,0.094118,0.349206,0.239437,0.349398,0.343719,0.163886,0.250816,0.415078,0.048272,0.272525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-18 16:00:00,0.707447,0.253521,0.097561,0.819892,0.704225,0.171975,0.848485,0.450704,0.031250,0.741784,0.591549,0.083333,0.721875,0.676056,0.152941,0.579365,0.281690,0.373494,0.194975,0.264475,0.267146,0.207106,0.061302,0.125695
2020-03-18 17:00:00,0.666667,0.244131,0.117886,0.736559,0.704225,0.163482,0.777778,0.469484,0.079861,0.701095,0.638498,0.079365,0.711458,0.676056,0.180392,0.497355,0.281690,0.365462,0.198995,0.228165,0.218158,0.211438,0.053306,0.130145
2020-03-18 18:00:00,0.625886,0.234742,0.138211,0.653226,0.704225,0.154989,0.707071,0.488263,0.128472,0.660407,0.685446,0.075397,0.701042,0.676056,0.207843,0.415344,0.281690,0.357430,0.230151,0.210500,0.185500,0.215771,0.054195,0.130145


In [ ]:
forecast_data = all_scaled_df.loc[str(pd.to_datetime('2019-03-18 12:00:00') - pd.to_timedelta(n_steps_in, unit = 'h')):]
forecast_index = []
forecast_X = []

for i in range(n_steps_in,len(forecast_data),6):
  forecast_index.append(forecast_data.index[i])

for ts in forecast_index:
  forecast_X.append(forecast_data.loc[str(ts - pd.to_timedelta(n_steps_in-1, unit='h')):str(ts)].values)
forecast_X = array(forecast_X)

In [ ]:
forecast_X.shape

(1466, 24, 24)

# Predict 72 hrs

In [ ]:
prediction_out = loaded_model.predict(forecast_X)
prediction_out

array([[[0.3321913 , 0.31650487, 0.27865806, 0.3312162 , 0.06907089,
         0.3173495 ],
        [0.32780555, 0.31129423, 0.2679667 , 0.34724715, 0.06950432,
         0.30943874],
        [0.3344477 , 0.30244514, 0.27890226, 0.336137  , 0.06824172,
         0.31432426],
        ...,
        [0.2810223 , 0.53039324, 0.26895788, 0.25718963, 0.08251007,
         0.27479178],
        [0.2810321 , 0.53132546, 0.2702544 , 0.2558835 , 0.0827563 ,
         0.26741144],
        [0.2740065 , 0.5218275 , 0.2596934 , 0.25360256, 0.08162948,
         0.26106298]],

       [[0.2865249 , 0.27758026, 0.23150119, 0.28852123, 0.06124523,
         0.28800145],
        [0.2820573 , 0.27042088, 0.22946233, 0.27976793, 0.05965436,
         0.29245657],
        [0.2824881 , 0.2647575 , 0.22310936, 0.2782091 , 0.05997322,
         0.29751292],
        ...,
        [0.2689147 , 0.42207465, 0.24491942, 0.2552904 , 0.07155724,
         0.2632196 ],
        [0.26735193, 0.41300935, 0.24108952, 0.25335956, 0.070

In [ ]:
prediction_out.shape

(1466, 72, 6)

In [ ]:
prediction_chan = prediction_out[:,:,0]
prediction_cm = prediction_out[:,:,1]
prediction_kan = prediction_out[:,:,2]
prediction_bkk = prediction_out[:,:,3]
prediction_kon = prediction_out[:,:,4]
prediction_song = prediction_out[:,:,5]

In [ ]:
prediction_chan_results =  pd.DataFrame(scaler[18].inverse_transform(prediction_chan))
prediction_chan_results.index = forecast_index
prediction_chan_results = prediction_chan_results.loc[:'2020-03-15 18:00:00']

prediction_chan_results.drop(prediction_chan_results[prediction_chan_results.index.isin(chan_test[chan_test['PM2.5'] == 0].index)].index, inplace=True)
prediction_chan_results

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
2019-03-18 12:00:00,33.053032,32.616653,33.277546,32.856026,31.851383,31.070347,29.635590,28.948120,28.464886,28.003242,28.129629,28.043640,27.347647,26.840225,25.595184,24.999292,24.537956,24.105650,24.688059,25.178015,25.872427,26.224039,25.991615,25.812643,27.599714,27.969492,28.781225,29.404196,29.594122,29.315779,28.747971,28.356415,27.867807,26.922258,26.658344,26.425064,26.404604,26.422897,26.289558,26.572487,26.898773,27.308075,27.472542,27.815819,28.146730,27.758446,27.997616,27.990822,27.645821,27.900642,27.836775,27.475685,27.642105,27.512146,27.909904,28.402861,28.501617,28.886124,29.122831,28.936703,28.358601,28.428179,27.860069,27.215803,28.242270,28.102970,28.374666,29.113197,28.752188,27.961720,27.962692,27.263645
2019-03-18 18:00:00,28.509226,28.064699,28.107567,27.594948,27.793060,27.552727,26.659418,25.960308,24.517714,23.905245,23.228361,22.891493,23.493639,23.954844,24.466253,24.838186,24.670414,24.560040,26.500858,26.987846,27.841404,28.815248,29.245804,29.644831,29.632446,28.933210,28.479870,27.420416,26.812729,26.532490,26.420177,26.328333,26.158199,26.346254,26.604425,26.800146,26.932692,27.386234,27.693390,26.827896,27.091040,26.898901,26.885744,27.348785,27.520912,27.710978,28.198811,28.290079,28.948475,29.580093,29.866148,30.225756,30.234688,29.993689,29.018291,29.261234,28.711332,28.203472,28.990334,28.673910,28.089781,29.159168,29.006399,28.469622,29.066069,28.693588,27.951406,27.906881,27.414347,26.757013,26.601517,26.306553
2019-03-19 00:00:00,26.781073,26.822151,26.131401,25.931358,25.577713,25.349489,25.501112,26.076513,26.825497,26.855154,26.650919,25.985460,27.608746,27.804052,28.637810,29.418385,29.698702,30.349716,30.546690,29.566534,28.932341,28.002499,27.620884,27.601454,27.908581,27.612461,27.280556,26.962378,26.805721,26.493769,26.321976,26.513554,26.888508,26.097187,26.458759,26.658327,27.823824,28.531994,28.934179,28.973001,29.271210,28.779467,28.581123,28.857496,28.757639,28.643497,28.583052,28.331804,28.171286,28.276649,27.869831,27.375429,28.154686,28.182756,28.111452,28.995640,29.047197,28.339622,28.617262,28.036644,26.867949,26.703348,26.157885,25.784742,26.017488,25.926424,26.322165,26.789368,26.719078,27.136923,27.365005,27.066624
2019-03-19 06:00:00,24.788671,25.630239,26.625418,27.030079,26.809469,26.217304,28.350931,28.773849,29.830257,30.204494,30.279953,30.440464,29.863813,28.950165,28.641676,28.330156,28.480436,28.687101,29.063482,28.980940,28.645988,28.271055,27.950518,27.686436,27.122192,26.772112,26.812832,26.612034,27.258276,28.153505,30.509544,31.073734,31.322636,30.830536,30.556519,29.616541,28.662790,28.427176,27.610058,27.453716,27.463224,27.519646,27.872301,28.432579,28.743372,28.937397,29.848122,30.271393,30.206718,31.220085,31.165707,30.283720,30.245831,29.476221,28.524855,28.157894,27.757242,27.726337,27.869967,27.903885,28.713425,29.221188,29.061604,29.070665,29.198299,28.370022,27.693657,27.771297,27.489082,27.754436,28.178652,28.578165
2019-03-19 12:00:00,25.130989,25.888933,27.250782,28.860022,29.085005,29.222033,28.473042,28.111107,28.063946,27.629602,27.710747,28.015594,28.086655,28.166742,27.670916,27.429106,27.413101,26.934582,26.365637,26.230673,26.464737,26.038958,25.946493,25.880049,28.107668,28.808859,30.158413,31.772947,31.773813,31.132439,29.775738,29.476536,29.303917,29.820507,30.890074,31.959242,33.322643,34.077065,34.728920,34.108776,34.677151,34.312069,33.894901,34.601032,34.585052,34.127628,33.988777,33.867290,33.885597,33.536301,33.167324,32.779423,32.403473,31.675020,31.145128,30.794907,29.726133,29.318922,29.474493,28.735811,27.972038,28.294767,27.942276,27.500164,28.366611,28.785608,29.112043,29.922213,30.158514,30.177523,30.740467,30.316664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
prediction_cm_results =  pd.DataFrame(scaler[19].inverse_transform(prediction_cm))
prediction_cm_results.index = forecast_index
prediction_cm_results = prediction_cm_results.loc[:'2020-03-15 18:00:00']

prediction_cm_results.drop(prediction_cm_results[prediction_cm_results.index.isin(cm_test[cm_test['PM2.5'] == 0].index)].index, inplace=True)
prediction_cm_results

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
2019-03-18 12:00:00,64.503693,63.441765,61.638321,60.270058,59.105915,58.780193,58.333958,56.863724,55.763924,51.430885,51.482082,51.105190,54.718288,57.962612,59.763882,61.143974,60.309586,57.963451,56.326088,56.586647,55.803333,56.996662,56.207413,53.910194,53.931229,54.575375,54.925171,60.316147,63.189625,66.093468,70.473808,71.564476,71.679703,70.406265,70.751900,70.141663,72.224640,74.232269,74.986038,77.072067,78.383141,78.024712,77.971565,78.401268,77.592644,78.334854,78.969200,78.614586,79.360344,79.090485,77.655884,76.321709,75.103844,72.823181,71.608299,71.461792,70.441910,71.545685,72.952820,73.887436,77.390663,80.559387,83.269272,88.770172,92.833038,95.313370,101.094070,104.417114,105.783142,108.094139,108.284126,106.348450
2019-03-18 18:00:00,56.570858,55.111774,53.957581,51.191448,52.111855,52.648338,56.539158,58.869205,59.621113,58.871964,56.874439,53.616562,51.807949,52.266102,51.949142,53.940983,53.306831,51.387253,52.625980,53.421242,54.030655,58.498642,61.004230,63.507828,67.778450,68.296425,67.557945,64.433449,63.156796,61.245350,61.448669,62.922150,63.396248,64.884766,64.789726,63.354359,61.139832,60.425236,59.309010,60.993736,62.799572,64.580109,69.370743,71.574677,72.986832,75.645462,76.359833,75.963203,75.779854,75.891808,75.003365,75.617813,77.165665,78.237785,81.634949,84.689674,86.772530,90.921043,93.867188,95.903152,100.484421,103.532478,105.112038,105.966522,105.352089,102.471497,97.648331,94.760773,90.253624,86.018814,84.171303,81.905952
2019-03-19 00:00:00,50.829308,53.614113,55.201126,56.525906,55.230675,52.680840,49.492416,49.518082,49.135632,51.672066,51.584419,50.017132,50.512554,50.176682,49.659546,52.390942,54.667770,56.952461,60.367111,59.837143,58.164665,54.357449,53.189003,51.731346,53.225311,54.981232,55.733360,56.370964,54.893414,52.268475,47.884548,46.075504,44.469360,45.562389,47.127453,48.789452,55.300873,59.083332,62.732796,69.600830,72.051430,73.933968,75.366585,75.804047,75.147942,75.855019,77.526627,78.203163,81.632034,83.905678,85.430130,89.389679,92.389610,94.368172,97.287262,97.937897,96.821068,94.134888,91.952667,88.199364,84.293571,82.091545,79.348557,78.649338,79.129845,79.041740,82.331886,85.240013,87.100838,92.038002,95.338760,97.507584
2019-03-19 06:00:00,48.210754,48.462032,47.990406,50.187462,49.412975,47.524403,49.220089,49.946026,50.823128,54.109814,56.509293,58.859604,59.591808,57.644394,54.675171,50.120361,50.222557,50.031097,53.611599,56.636673,58.174294,58.822777,56.477608,52.838524,47.030106,44.531384,42.362129,43.589600,45.325687,46.709618,54.237602,58.140274,61.915779,68.181442,69.359947,70.314171,69.793419,69.038475,67.378052,67.596893,68.999573,69.845390,73.189629,76.190773,78.235069,82.385994,84.144081,84.725571,85.660812,85.499817,83.758781,82.773048,80.772644,77.741547,75.968460,74.069725,71.792046,72.435974,73.096596,73.228317,77.184357,80.129784,82.210518,89.050926,92.673790,95.452522,99.113480,100.317146,99.979225,98.280083,98.254829,96.604691
2019-03-19 12:00:00,44.828564,46.460205,47.727520,52.158627,53.579578,55.463242,57.388168,56.956993,56.189034,51.248852,49.980160,48.206013,48.133553,50.671341,52.546803,56.660744,57.800068,57.425697,54.676315,53.432186,51.808044,53.513847,54.454594,54.399567,57.708942,60.395142,62.176483,68.796806,70.470703,71.770004,70.337326,68.189590,65.945694,61.213882,62.498150,63.250385,67.783661,72.622322,75.774498,79.760323,79.924850,78.515366,76.373070,74.230591,71.056076,69.684166,67.565613,64.848846,66.238892,66.045113,66.110817,70.790718,72.301262,73.861702,79.133217,81.946762,84.204002,90.666031,94.220306,97.046715,101.770729,103.846336,105.021851,105.336945,106.280823,105.477554,105.058838,106.514832,105.908905,108.484795,110.547897,110.848381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
prediction_kan_results =  pd.DataFrame(scaler[20].inverse_transform(prediction_kan))
prediction_kan_results.index = forecast_index
prediction_kan_results = prediction_kan_results.loc[:'2020-03-15 18:00:00']

prediction_kan_results.drop(prediction_kan_results[prediction_kan_results.index.isin(kan_test[kan_test['PM2.5'] == 0].index)].index, inplace=True)
prediction_kan_results

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
2019-03-18 12:00:00,42.662548,41.025700,42.699936,41.633987,40.058727,39.120411,36.669563,36.329185,34.581654,33.395077,34.357136,33.349888,32.721416,32.529804,31.541935,31.881124,32.187500,31.053677,31.863691,32.030415,33.207405,33.762012,32.766041,32.866005,35.200817,35.053562,37.072159,38.369179,38.230877,38.999084,38.754051,38.241924,37.545647,36.141411,35.793209,35.222279,35.621536,35.657372,35.727779,36.774712,37.464901,37.917969,38.478218,38.798698,39.091068,38.733364,38.607780,38.685303,38.454781,38.545128,38.098171,37.630722,37.659855,36.739620,37.395592,38.102913,37.606415,38.564884,38.451488,38.002144,37.693600,37.155930,36.766796,37.032673,38.840233,38.948292,40.753559,41.348656,41.280972,41.177452,41.375950,39.759060
2019-03-18 18:00:00,35.442833,35.130684,34.158043,32.978897,33.897350,33.092545,32.334839,31.913324,30.372425,30.463373,30.324245,29.080545,29.768291,29.610779,30.900934,31.644474,30.589355,31.076910,33.866730,33.946541,36.153305,37.926559,38.091293,39.637321,39.520576,38.124004,37.586678,35.490784,34.976151,33.837643,33.697906,33.649559,33.583508,34.512108,35.993664,35.643452,36.071163,36.732349,36.792477,35.635815,35.022259,35.592186,35.970352,35.901226,36.697121,37.396000,38.011726,37.760895,38.938850,39.881706,39.402309,40.030502,39.352669,38.851078,37.957836,38.368015,37.769978,38.351498,39.757072,39.369267,40.056873,41.542576,41.507328,41.757812,42.939732,41.364182,40.551159,40.555260,38.552906,37.497162,36.910805,35.732128
2019-03-19 00:00:00,32.403908,32.788277,32.511074,33.344841,33.716835,32.843090,32.789413,32.934303,34.216991,34.501072,33.572052,33.384995,35.799553,35.682487,37.362576,38.309826,37.861137,39.799011,39.648327,37.862457,36.926018,34.747784,34.595871,33.833633,34.260273,33.885540,33.753761,33.927979,35.397751,33.962242,33.602226,33.597130,33.735355,32.266727,31.599152,32.931278,34.870655,34.911373,36.948242,37.848835,38.241558,37.946468,38.030815,38.447586,37.639725,37.629696,36.988476,36.793335,37.140400,37.133354,36.839596,37.325478,38.462696,38.897243,39.895309,41.275894,40.916718,40.452309,41.057522,39.256153,37.712940,37.017223,35.623703,35.098843,34.718639,34.583443,35.658096,35.451096,35.911568,37.345051,36.849930,36.926567
2019-03-19 06:00:00,31.510813,31.873787,33.723583,34.630867,33.711876,33.537018,36.641449,36.769382,38.966381,39.586487,39.253948,40.245953,38.818878,37.640728,36.122219,34.806297,35.667950,35.224968,36.017162,35.999519,36.184948,36.536209,37.691975,36.015945,34.535759,33.308277,33.048847,32.178829,31.534973,34.112473,37.567463,37.288155,39.443970,39.592518,39.490055,38.246166,37.071121,37.459717,35.344032,35.180717,34.718357,35.007488,36.084549,36.887672,37.946308,39.289856,40.556892,41.435635,42.061123,43.334347,43.275265,42.525864,41.788174,40.651390,39.069935,37.358765,36.786888,36.483349,35.615517,36.036015,37.694027,37.636936,38.122807,39.330261,38.678337,38.365776,38.273281,38.317398,37.544128,38.286289,39.358139,39.050179
2019-03-19 12:00:00,31.704817,31.479910,34.889664,37.375698,37.446644,38.206734,37.211834,36.719700,35.867069,34.252701,35.331459,34.162323,34.426720,35.003204,34.478859,35.244839,36.350113,35.465019,34.815659,33.984581,33.920986,32.919422,31.321939,32.081173,35.149349,34.824154,38.045094,40.792404,41.355591,40.605011,38.702358,39.575706,37.276199,37.722630,39.720093,40.361210,42.446079,43.276264,45.135986,45.596420,47.461411,46.732487,46.636433,46.960358,46.999023,46.019993,44.419781,44.853977,44.421318,42.214577,42.856094,42.247112,40.444225,40.471760,40.131420,39.047760,38.462543,39.161255,39.273376,39.028275,39.289494,39.947987,39.324921,39.467506,41.054802,40.924503,41.861584,42.804005,43.096462,44.027721,44.521671,44.412449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
prediction_bkk_results =  pd.DataFrame(scaler[21].inverse_transform(prediction_bkk))
prediction_bkk_results.index = forecast_index
prediction_bkk_results = prediction_bkk_results.loc[:'2020-03-15 18:00:00']

prediction_bkk_results.drop(prediction_bkk_results[prediction_bkk_results.index.isin(bkk_test[bkk_test['PM2.5'] == 0].index)].index, inplace=True)
prediction_bkk_results

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
2019-03-18 12:00:00,38.222347,40.072323,38.790211,38.713196,37.156204,35.992134,34.413872,32.692459,32.227062,32.441746,32.337345,31.588306,30.831007,29.720812,27.760878,27.837399,27.694017,27.404135,28.803772,30.397532,30.270206,30.503403,31.280876,29.598471,32.463837,34.822403,34.221996,35.368656,35.733055,34.716244,34.087761,33.068989,32.356850,31.022852,30.773403,30.143286,30.183264,30.517027,29.849566,30.578007,31.383919,31.689959,32.279587,32.693546,33.052708,32.533783,32.520714,32.527599,32.050102,32.091587,32.126881,31.634655,31.770861,31.530767,32.249458,33.085815,33.116383,33.701805,33.610935,33.310070,32.023361,31.220936,30.628174,29.380075,30.311848,30.908766,30.920399,31.552769,31.326353,29.679684,29.528957,29.265736
2019-03-18 18:00:00,33.295349,32.285217,32.105328,31.987331,31.839350,31.101973,29.932426,28.442055,26.485437,26.491222,26.168867,26.040648,27.288780,28.748247,28.464594,28.595837,29.693678,28.091476,31.186068,33.697060,33.337543,35.144169,36.207550,35.791576,35.427685,34.181374,32.791256,31.390783,30.838249,30.039860,30.295481,30.430372,29.417927,30.440077,30.797770,31.644871,32.765377,32.872108,33.688656,32.390232,32.154854,31.660835,31.366203,31.929373,31.581602,31.830145,32.351017,32.360744,33.472778,34.277405,34.520397,34.819481,34.261536,33.590076,31.737818,31.206470,31.130594,30.506214,30.527712,31.432066,30.498230,31.013817,32.029095,31.005466,31.122602,31.793026,30.664726,30.428581,30.517084,29.460510,29.237692,29.151947
2019-03-19 00:00:00,31.044220,30.749567,29.678734,30.048447,29.931133,30.223663,30.942745,32.238121,32.625942,32.128242,32.608253,30.655519,33.378586,35.534798,35.248947,36.610397,37.663532,36.997463,36.806705,35.317875,33.471966,32.489639,32.568321,31.656109,32.579903,32.311260,30.778439,31.296051,30.733082,31.571911,32.403152,32.207951,32.884670,31.536274,32.072800,31.328596,32.818340,34.121868,33.506470,33.654247,33.373108,32.860107,32.708328,32.806175,32.470936,32.054901,31.874731,31.061508,30.598257,30.333941,30.084444,29.472954,30.018826,30.947815,30.772375,31.250340,32.437847,31.422062,31.149431,31.634491,29.922266,29.469395,29.282398,28.410748,28.912766,28.555769,28.605721,29.557451,28.830582,28.824911,29.233160,28.326408
2019-03-19 06:00:00,29.757072,31.767160,32.261894,32.316723,32.982590,31.229918,34.654228,37.064137,36.967373,37.831501,37.892231,37.273739,36.112022,34.588272,33.484932,33.629097,34.114166,33.659611,34.499741,34.414097,32.922039,33.475536,32.711678,33.595119,33.451145,32.380337,32.424870,31.675978,33.755547,32.786087,35.901787,37.360950,36.079323,35.705154,34.568127,33.624298,32.684151,31.505451,31.103998,30.809570,31.060078,30.436966,30.917376,31.820839,32.271084,32.784130,33.508614,34.949722,34.889668,35.741879,36.580627,35.242054,34.967049,34.404182,32.727520,32.527206,31.498045,31.032984,31.856367,30.897652,31.813032,32.821003,31.951536,31.590979,31.315763,30.126221,28.699631,28.721256,28.381510,28.791206,29.752829,30.420073
2019-03-19 12:00:00,30.052588,33.462357,33.109180,35.590298,35.956985,35.794685,34.920441,33.743042,33.426052,32.989422,32.862595,33.069054,33.855831,33.309982,32.414894,32.653114,32.224422,32.392590,32.274364,31.898716,31.941341,30.495874,31.091749,29.258707,32.072414,35.482494,34.407993,36.841801,36.505749,35.795803,34.556072,32.917664,33.898079,35.277058,36.729553,37.633068,39.331539,40.212391,40.309902,39.987461,39.636242,41.172970,41.155369,41.764938,42.156101,41.181881,41.749630,40.591187,40.405464,40.478584,38.636265,37.826286,37.478027,35.194241,34.338009,33.710484,31.626694,31.238117,31.144678,30.423126,29.494711,29.044779,29.665018,29.057501,29.824297,31.142357,31.409283,32.586388,32.973663,32.795349,33.610626,33.358269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
prediction_kon_results =  pd.DataFrame(scaler[22].inverse_transform(prediction_kon))
prediction_kon_results.index = forecast_index
prediction_kon_results = prediction_kon_results.loc[:'2020-03-15 18:00:00']

prediction_kon_results.drop(prediction_kon_results[prediction_kon_results.index.isin(kon_test[kon_test['PM2.5'] == 0].index)].index, inplace=True)
prediction_kon_results

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
2019-03-18 12:00:00,54.420956,54.762455,53.767647,53.017902,51.771049,51.138462,49.911896,48.394104,48.211567,46.299294,46.454304,46.281490,46.875656,48.035820,47.068447,46.581219,46.055500,44.685661,44.486397,45.592270,45.393291,45.943420,46.158806,44.574947,46.008503,47.811787,47.440350,49.732971,51.678204,51.805157,52.646381,52.718777,52.589733,51.187023,51.209766,50.916325,51.370739,52.585072,52.424759,53.053928,54.146488,54.202621,54.008736,54.607098,54.720318,54.419060,55.062492,55.017872,54.740265,55.011627,54.612080,53.606556,53.505630,52.748348,52.432316,53.265171,52.822208,53.290180,54.406700,54.599129,55.161625,56.748169,57.262733,58.270271,61.123100,61.496185,63.143383,65.445335,65.513184,65.009682,65.203682,64.315872
2019-03-18 18:00:00,48.255119,47.001667,47.252903,45.849358,46.454765,46.351883,46.830723,47.295029,46.027107,44.816559,43.564342,42.142120,41.997467,43.187141,42.898846,43.748188,44.212997,42.625637,44.599308,46.526566,46.240417,48.489704,50.456757,51.067715,52.514343,52.032948,51.733761,49.734272,48.780571,48.055923,47.931801,48.888760,48.499226,48.900139,49.160027,48.725864,47.818924,47.820309,47.962990,47.759666,49.008621,49.296478,50.848915,52.627598,52.885933,53.770828,54.797264,54.607307,54.862049,55.774227,55.692535,56.057003,56.964657,57.298042,57.519253,59.158966,59.343636,59.939400,61.842850,62.342377,62.873043,65.033142,65.605690,64.786919,64.796562,64.166939,61.276634,60.015739,58.735600,56.379948,55.693790,55.084805
2019-03-19 00:00:00,44.899273,46.237617,45.755089,45.600086,44.786697,43.716999,42.720272,43.521885,43.797543,44.516682,44.963593,43.140549,44.526039,45.614952,45.176708,46.812656,48.596317,49.391342,50.785492,49.696850,48.877426,46.867886,46.412186,45.750519,46.488125,47.422016,46.917549,46.630344,45.732899,44.657303,42.872181,42.153564,42.294262,42.129189,43.584316,43.816441,47.085686,50.082699,50.686924,52.948647,54.472027,54.410431,54.544735,55.157551,54.919128,54.910511,55.983257,55.838821,56.701126,58.015266,58.070335,58.640911,60.729885,61.308357,61.744141,62.588474,62.675858,60.563713,59.679199,58.569435,55.812267,54.899269,53.963833,53.195992,54.060482,53.886410,55.087551,57.369812,57.504440,59.209244,61.367638,61.580013
2019-03-19 06:00:00,41.782265,43.044540,43.272446,44.127613,44.308460,42.447361,44.739540,46.446404,46.635067,48.059906,49.391766,50.087818,49.933319,48.032856,47.466606,45.705223,46.169262,46.060287,47.534271,49.166039,48.800014,48.343117,46.827927,45.710854,43.232311,41.707481,41.589516,42.026619,44.281475,44.475693,49.133034,52.292984,52.522388,54.117126,54.531609,53.965778,52.840748,52.187599,51.262913,51.131172,52.139553,52.252262,53.518764,55.624966,56.142319,57.371819,58.861782,59.489559,59.275822,60.050056,59.566917,57.957848,57.353092,55.867893,54.239922,53.709354,52.651791,52.776329,54.006168,53.628017,55.442024,57.961456,57.843140,59.852562,62.054394,61.959503,62.280258,63.087959,62.882446,62.082157,62.541782,62.549686
2019-03-19 12:00:00,40.698177,43.406670,43.494473,46.385288,47.476482,47.910835,47.960480,47.133308,47.468136,45.502857,44.846981,44.905323,44.846016,46.038506,46.021824,46.934204,47.366608,46.661755,45.061615,44.407070,44.419552,44.765987,45.824417,45.106247,48.114201,51.449585,51.558929,55.120636,56.120071,55.683800,53.973907,52.167622,52.341778,51.004963,52.457371,53.663097,56.328114,59.250202,60.404129,60.792156,61.017296,60.439388,58.892033,58.679855,57.747532,56.687965,56.437881,55.030739,55.568214,56.176048,55.247883,56.592041,57.836407,57.068817,58.374218,60.046112,59.238262,60.799004,63.053085,62.827827,63.378841,64.478378,64.906029,64.150833,65.075279,65.656601,65.242844,66.680901,66.766891,67.069832,68.882942,68.325470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
prediction_song_results =  pd.DataFrame(scaler[23].inverse_transform(prediction_song))
prediction_song_results.index = forecast_index
prediction_song_results = prediction_song_results.loc[:'2020-03-15 18:00:00']

prediction_song_results.drop(prediction_song_results[prediction_song_results.index.isin(song_test[song_test['PM2.5'] == 0].index)].index, inplace=True)
prediction_song_results

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
2019-03-18 12:00:00,28.529720,27.818542,28.257751,28.246166,27.524527,27.484838,27.322014,27.311434,27.527315,27.158222,27.238144,27.553055,27.395920,25.864279,25.576931,23.782833,22.298513,21.495401,21.501530,21.347218,22.061190,22.836300,22.569202,22.855488,23.572353,22.468134,23.377274,23.385166,22.156488,22.790264,22.654482,22.358595,22.344221,22.693464,22.400661,22.312857,22.468634,21.625114,21.729849,21.685343,20.982367,21.085896,21.256025,21.016083,21.304546,21.814184,21.541876,21.971130,22.405918,22.266449,22.629078,23.034199,23.107800,23.494867,23.771864,23.494846,23.998158,24.164013,24.241856,24.993572,25.535057,25.474689,26.160370,25.987705,24.776043,24.935459,24.900425,24.373791,24.627295,24.703781,24.040289,23.469563
2019-03-18 18:00:00,25.891331,26.291845,26.746412,26.685944,26.443890,26.752871,26.504738,25.174395,24.582428,22.874235,21.756069,20.862080,21.111170,20.943188,21.656250,22.279737,21.826319,22.079664,22.529148,21.273001,22.008898,21.895061,21.056089,21.828068,23.239653,23.638573,23.779608,24.456276,24.109512,24.041819,24.029659,23.341854,23.657637,22.978121,21.699757,21.316257,20.636374,20.508400,20.754189,21.141165,20.918709,21.795912,22.569416,22.357725,23.370583,23.772577,23.622622,24.289047,24.586950,24.785862,25.497686,26.287355,26.858259,27.737980,28.273571,27.549047,27.850904,27.019928,26.093184,26.107676,25.308075,24.607290,24.661053,24.496290,24.407780,23.898163,23.850903,23.896151,23.377188,23.663441,23.699753,23.658369
2019-03-19 00:00:00,25.337820,24.305389,24.031729,22.588686,21.387941,20.433510,20.487652,20.635380,21.301643,21.976351,21.350487,21.341909,21.497736,20.405544,20.950294,21.477795,21.285345,22.438488,24.319231,24.760889,24.970695,25.532490,25.119104,25.474018,25.571966,24.994024,25.453758,24.357109,22.715300,21.761374,20.925280,21.279041,21.644726,22.443684,22.450464,23.612280,24.627148,23.904743,25.354561,25.216015,24.402668,25.029810,25.095087,25.304272,25.844526,26.512104,26.508078,27.114025,27.457777,26.772963,26.996422,26.260687,25.251919,25.381886,25.147255,24.912453,24.722927,24.416494,24.147087,23.480892,23.288725,23.186605,23.040615,23.646456,23.563490,24.066162,24.789072,24.550711,25.414186,26.046034,25.867821,26.622704
2019-03-19 06:00:00,20.738436,20.906471,21.625565,22.148804,21.402878,21.244278,21.595924,20.566593,21.357874,21.470032,21.162743,21.994884,23.585766,24.533794,24.651413,25.095793,24.689640,25.209108,25.471386,24.686586,25.107428,23.655012,22.377329,21.368547,21.465403,22.261032,22.637548,24.105965,24.273100,25.903984,27.522907,26.524071,28.053963,27.366907,26.334244,26.607040,26.145653,25.878283,25.570505,25.928440,25.921480,26.422441,26.610958,25.629904,25.921844,25.451241,24.833790,24.973633,24.843588,24.320391,24.176117,23.873560,23.346432,23.223747,23.905134,23.933268,24.456667,25.650246,25.498857,26.519291,27.115412,26.447948,27.490526,27.452452,26.794605,27.524141,27.678877,27.568069,27.628025,27.777220,27.603535,27.266083
2019-03-19 12:00:00,20.876450,20.116186,21.301147,21.776516,21.132198,21.688919,21.688944,22.564880,23.028460,24.052221,24.518940,24.738819,24.240713,23.630217,23.814419,22.789949,21.251047,20.650122,20.037048,20.764524,21.268194,22.990889,23.433741,24.470577,26.145365,25.638000,27.403463,28.012548,26.641293,26.805946,26.092537,26.454550,26.160149,26.049570,26.661619,27.336527,28.827253,28.594955,29.494102,28.353411,26.712145,25.958788,25.147070,25.046366,25.073709,25.985594,25.672459,26.283976,27.530287,27.028772,28.302036,29.206934,28.589933,29.903456,30.180172,29.051323,29.854212,29.029610,27.430538,27.797346,26.860882,26.144989,26.015268,25.803755,25.766813,25.536737,26.108559,25.784491,25.807642,25.981216,24.596928,24.648460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# output to csv

In [ ]:
out_val = np.vstack((prediction_chan_results.values.reshape(-1, 1),
                    prediction_cm_results.values.reshape(-1, 1),
                    prediction_kan_results.values.reshape(-1, 1),
                    prediction_bkk_results.values.reshape(-1, 1),
                    prediction_kon_results.values.reshape(-1, 1),
                    prediction_song_results.values.reshape(-1, 1)
                    ))

In [ ]:
out_df = pd.DataFrame(out_val)
out_df = out_df.rename(columns={0:'Predicted'})
out_df.index.name = 'id'
out_df

,Predicted
id,
0,33.053032
1,32.616653
2,33.277546
3,32.856026
4,31.851383
...,...
481315,21.102125
481316,22.669689
481317,22.826767


In [ ]:
# path = '/content/drive/Shareddrives/DS/lstm_out/lstm_out_2.csv'
# out_df.to_csv(path)

##out to submission

In [ ]:
your_submission = out_df.copy(True)

In [ ]:
miss_index = pd.read_csv('/content/drive/Shareddrives/DS/DS_kaggle_edited/missing_index.csv')
prediction_index = pd.read_csv('/content/drive/Shareddrives/DS/DS_kaggle_edited/output_index.csv')

In [ ]:
your_submission['datetime index'] = prediction_index['predicted timestamp']
your_submission

,Predicted,datetime index
id,,
0,33.053032,Chanthaburi 2019-03-18 13:00:00
1,32.616653,Chanthaburi 2019-03-18 14:00:00
2,33.277546,Chanthaburi 2019-03-18 15:00:00
3,32.856026,Chanthaburi 2019-03-18 16:00:00
4,31.851383,Chanthaburi 2019-03-18 17:00:00
...,...,...
481315,21.102125,Songkhla 2020-03-18 14:00:00
481316,22.669689,Songkhla 2020-03-18 15:00:00
481317,22.826767,Songkhla 2020-03-18 16:00:00


In [ ]:
cond = your_submission['datetime index'].isin(miss_index['Missing date'])
your_submission.drop(your_submission[cond].index, inplace = True)

In [ ]:
your_submission.reset_index(inplace=True)
your_submission.drop(columns={'datetime index'},inplace=True)
your_submission

,id,Predicted
0,0,33.053032
1,1,32.616653
2,2,33.277546
3,3,32.856026
4,4,31.851383
...,...,...
448549,481315,21.102125
448550,481316,22.669689
448551,481317,22.826767
448552,481318,21.943460


In [ ]:
your_submission['id'] = your_submission.index
your_submission = your_submission[['Predicted']]
your_submission

,Predicted
0,33.053032
1,32.616653
2,33.277546
3,32.856026
4,31.851383
...,...
448549,21.102125
448550,22.669689
448551,22.826767
448552,21.943460


In [ ]:
your_submission.index.name = 'id'
your_submission

,Predicted
id,
0,33.053032
1,32.616653
2,33.277546
3,32.856026
4,31.851383
...,...
448549,21.102125
448550,22.669689
448551,22.826767


In [ ]:
your_submission.to_csv('/content/drive/Shareddrives/DS/lstm_out_4.csv') #To your prefer path